In [1]:

%pprint

Pretty printing has been turned OFF



---
# Load needed libraries and functions

In [2]:

from datetime import datetime
import humanize
import os
import re
import shutil
import sys
import time
import warnings
import winsound

warnings.filterwarnings('ignore')
duration = 1000  # milliseconds
freq = 880  # Hz

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

t0 = time.time()

# Get the Neo4j driver
from storage import Storage
s = Storage(
    data_folder_path=os.path.abspath('../data'),
    saves_folder_path=os.path.abspath('../saves')
)

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(
    s=s,
    secrets_json_path=os.path.abspath('../data/secrets/jh_secrets.json')
)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

# Get the neo4j object
from cypher_utils import CypherUtilities
cu = CypherUtilities(
    uri=uri, user=user, password=password, driver=None, s=s, ha=ha
)

from section_utils import SectionUtilities
su = SectionUtilities(s=s, ha=ha, wsu=wsu, cu=cu, crf=None, verbose=False)

duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
print(f'Utility libraries created in {duration_str}')
print(f'Last run on {datetime.now()}')

Utility libraries created in 9 seconds
Last run on 2023-03-16 12:11:38.409148



----

In [4]:

file_path = r'C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\data\html\other_email.html'
file_name = re.sub(r'[^A-Za-z0-9]+', ' ', '''
    1140-1
    3/16/2023 - 3/15/2024
    Remote
    Professional - US - Data Scientist IV
    ''').strip().replace(' ', '_') + '.html'
new_file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
if os.path.isfile(new_file_path):
    file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}'
    new_file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
if not os.path.isfile(new_file_path):
    shutil.copy(file_path, os.path.join(cu.SAVES_HTML_FOLDER, file_name))
    page_soup = wsu.get_page_soup(file_path)
    div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
    child_strs_list = ha.get_navigable_children(div_soup, [])
    cu.ensure_filename(file_name, verbose=False)
    cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
    print(file_name)

1140_1_3_16_2023_3_15_2024_Remote_Professional_US_Data_Scientist_IV.html


In [5]:

# Add the posting URL to the file name only if you have one
posting_url = ''
if posting_url:
    cypher_str = f'''
        MATCH (fn:FileNames {{file_name: "{file_name}"}})
        SET fn.posting_url = "{posting_url}"
        RETURN fn;'''
    with cu.driver.session() as session:
        row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
    display(row_objs_list)

In [6]:

# Check if the lru has retrained its is-header classifier
t1 = time.time()
from hc_utils import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=False)
from lr_utils import LrUtilities
lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
if not hasattr(lru, 'ISHEADER_PREDICT_PERCENT_FIT'):
    if not hasattr(lru, 'ISHEADER_LR'):
        lru.build_isheader_logistic_regression_elements(verbose=True)
    lru.retrain_isheader_classifier(verbose=True)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f')
print(f'Is-header classifier retrained in {duration_str}')

I have 81,693 hand-labeled header htmls in here
Is-header classifier retrained in 29 seconds


In [7]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
is_header_list = []
for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
    if is_header is None:
        probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
        idx = probs_list.index(max(probs_list))
        is_header = [True, False][idx]
    is_header_list.append(is_header)
feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_tuple_list = []
for feature_dict in feature_dict_list:
    feature_tuple_list.append(hc.get_feature_tuple(feature_dict, pos_lr_predict_single=None, pos_crf_predict_single=None))
from crf_utils import CrfUtilities
crf = CrfUtilities(ha=ha, hc=hc, cu=cu, lru=lru, verbose=True)
crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))

In [117]:

db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)
winsound.Beep(freq, duration)

['H-O', 'O-RQ', 'O-RQ', 'O-IP', 'O-JT', 'O-JD', 'O-OL', 'O-JT', 'H-TS', 'O-JT', 'O-TS', 'H-TS', 'O-TS', 'O-TS', 'O-TS', 'H-RQ', 'O-RQ', 'H-RQ', 'H-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'H-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'O-PQ', 'H-O', 'O-TS', 'O-O', 'O-CS', 'O-O', 'O-O', 'O-TS', 'O-O', 'O-CS', 'O-IP', 'O-CS', 'O-IP', 'O-TS', 'O-SP', 'O-CS', 'O-TS', 'O-OL']
[1, 2, 16, 19, 20, 21, 22, 23, 24, 25]


[1, 2, 16, 19, 20, 21, 22, 23, 24, 25]



----

In [ ]:
raise

In [114]:

# Display the context of an individual child string
idx = 34
print(indices_list); child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]; basic_quals_dict = s.load_object('basic_quals_dict')
print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end='')
print(f'{idx} {pos_symbol}) {child_str}')

[1, 2, 3, 16, 19, 21, 22, 23, 24, 25, 30, 32, 33]
34 O-O) <span>--</span>


In [112]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[child_str] = 1
s.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"•             Degree in Quantitative Discipline (e.g. physics, engineering, computer science, applied mathematics, etc.)" in basic_quals_dict: 1


In [116]:

def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = """MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        SET
            np.is_header = 'True',
            np.is_task_scope = 'False',
            np.is_minimum_qualification = 'False',
            np.is_preferred_qualification = 'False',
            np.is_educational_requirement = 'False',
            np.is_legal_notification = 'False',
            np.is_other = 'True',
            np.is_corporate_scope = 'False',
            np.is_job_title = 'False',
            np.is_office_location = 'False',
            np.is_job_duration = 'False',
            np.is_supplemental_pay = 'False',
            np.is_interview_procedure = 'False',
            np.is_posting_date = 'False'
        """ + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
row_objs_list

[{'navigable_parent': '<span>--</span>', 'is_header': 'True', 'is_task_scope': 'False', 'is_qualification': None, 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'True'}]

In [115]:

# Show what's in the database already for this html string
def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = '''
        MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        ''' + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=False)
row_objs_list

[{'navigable_parent': '<span>--</span>', 'is_header': None, 'is_task_scope': None, 'is_qualification': None, 'is_minimum_qualification': None, 'is_preferred_qualification': None, 'is_legal_notification': None, 'is_job_title': None, 'is_office_location': None, 'is_job_duration': None, 'is_supplemental_pay': None, 'is_educational_requirement': None, 'is_interview_procedure': None, 'is_corporate_scope': None, 'is_posting_date': None, 'is_other': None}]

In [ ]:

# Remove this particular child string from the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = child_strs_list[idx]
basic_quals_dict.pop(child_str)
# basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')


----